<a href="https://colab.research.google.com/github/joseghm1/AIHW/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()

        # Define various layers here
        self.fc1 = nn.Linear(28*28, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 10)



        # Layers for model_3 (deep network, >4 layers, >200 neurons)
        self.deep_fc1 = nn.Linear(28 * 28, 512)
        self.deep_fc2 = nn.Linear(512, 512)
        self.deep_fc3 = nn.Linear(512, 512)
        self.deep_fc4 = nn.Linear(512, 512)
        self.deep_fc5 = nn.Linear(512, 10)

        # This will select the forward pass function based on mode for the ConvNet.
        # During creation of each ConvNet model, you will assign one of the valid mode.
        # This will fix the forward function (and the network graph) for the entire training/testing
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        else:
            print("Invalid mode ", mode, "selected. Select between 1-3")
            exit(0)

    # Baseline sample model
    def model_0(self, X):
        # ======================================================================
        # Three fully connected layers with activation

        X = torch.flatten(X, start_dim=1)
        X = self.fc1(X)
        X = F.relu(X)
        X = self.fc2(X)
        X = F.relu(X)
        X = self.fc3(X)
        X = torch.sigmoid(X)

        return X

    # Baseline model. task 1
    def model_1(self, X):
        # ======================================================================
        # Three fully connected layers without activation

        X = torch.flatten(X, start_dim=1)
        # NotImplementedError
        X = self.fc1(X)
        X = self.fc2(X)
        X = self.fc3(X)
        return X


    # task 2
    def model_2(self, X):
        # ======================================================================
        # Train with activation (use model 1 from task 1)

        X = torch.flatten(X, start_dim=1)
        # NotImplementedError

        X = self.fc1(X)
        X = F.relu(X)
        X = self.fc2(X)
        X = F.relu(X)
        X = self.fc3(X)

        return X


    # task 3
    def model_3(self, X):
        # ======================================================================
        # Change number of fully connected layers and number of neurons from model 2 in task 2

        X = torch.flatten(X, start_dim=1)
        X = self.deep_fc1(X)
        X = F.relu(X)
        X = self.deep_fc2(X)
        X = F.relu(X)
        X = self.deep_fc3(X)
        X = F.relu(X)
        X = self.deep_fc4(X)
        X = F.relu(X)
        X = self.deep_fc5(X)
        return X



In [14]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
#from ConvNet import ConvNet
import argparse
import numpy as np


def train(model, device, train_loader, optimizer, criterion, epoch, batch_size):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''

    # Set model to train mode before each epoch
    model.train()

    # Empty list to store losses
    losses = []
    correct = 0

    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        data, target = batch_sample

        # Push data/label to correct device
        data, target = data.to(device), target.to(device)

        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()

        # Do forward pass for current set of data
        output = model(data)

        # ======================================================================
        # Compute loss based on criterion
        # ----------------- YOUR CODE HERE ----------------------
        #
        # Remove NotImplementedError and assign correct loss function.
        loss = criterion(output, target)


        # Computes gradient based on final loss
        loss.backward()

        # Store loss
        losses.append(loss.item())

        # Optimize model parameters based on learning rate and gradient
        optimizer.step()

        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)

        # ======================================================================
        # Count correct predictions overall
        correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    return train_loss, train_acc



def test(model, device, test_loader):
    '''
    Tests the model.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    test_loader: dataloader for test samples.
    '''

    # Set model to eval mode to notify all layers.
    model.eval()

    losses = []
    correct = 0

    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample
            data, target = data.to(device), target.to(device)


            # Predict for data by doing forward pass
            output = model(data)

            # ======================================================================
            # Compute loss based on same criterion as training
            loss = F.cross_entropy(output, target, reduction='mean')

            # Append loss to overall test loss
            losses.append(loss.item())

            # Get predicted index by selecting maximum log-probability
            pred = output.argmax(dim=1, keepdim=True)

            # ======================================================================
            # Count correct predictions overall
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))

    return test_loss, accuracy




def run_main(FLAGS):
    # Check if cuda is available
    use_cuda = torch.cuda.is_available()

    # Set proper device based on cuda availability
    device = torch.device("cuda" if use_cuda else "cpu")
    print("Torch device selected: ", device)

    # Create transformations to apply to each data sample
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    # Load datasets for training and testing
    dataset1 = datasets.MNIST('./data/', train=True, download=True, transform=transform)
    dataset2 = datasets.MNIST('./data/', train=False, transform=transform)
    train_loader = DataLoader(dataset1, batch_size=FLAGS.batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset2, batch_size=FLAGS.batch_size, shuffle=False, num_workers=4)

    # Dictionary to store results for each mode
    results = {}

    # === FOR LOOP STARTS HERE ===
    # Loop over modes 1, 2, and 3
    for mode in [1, 2, 3]:
        print(f"\n=== Starting Training for Mode {mode} ===")

        # Initialize the model for the current mode and send to device
        model = ConvNet(mode).to(device)

        # Initialize the criterion for loss computation
        criterion = nn.CrossEntropyLoss()

        # Initialize optimizer type
        optimizer = optim.SGD(model.parameters(), lr=FLAGS.learning_rate, weight_decay=1e-7)

        best_accuracy = 0.0

        # Run training for n_epochs specified in config
        for epoch in range(1, FLAGS.num_epochs + 1):
            print(f"\nEpoch: {epoch} (Mode {mode})")
            train_loss, train_accuracy = train(model, device, train_loader,
                                               optimizer, criterion, epoch, FLAGS.batch_size)
            test_loss, test_accuracy = test(model, device, test_loader)

            if test_accuracy > best_accuracy:
                best_accuracy = test_accuracy

        # Store results for this mode
        results[mode] = best_accuracy
        print(f"\nMode {mode} - Best Test Accuracy: {best_accuracy:.2f}%")
    # === FOR LOOP ENDS HERE ===

    # Print final comparison of all modes
    print("\n=== Final Results ===")
    for mode, accuracy in results.items():
        print(f"Mode {mode}: Best Test Accuracy = {accuracy:.2f}%")
    print(f"Best Overall Mode: {max(results, key=results.get)} with {results[max(results, key=results.get)]:.2f}%")

if __name__ == '__main__':
    # Set parameters
    parser = argparse.ArgumentParser('CNN Exercise.')
    parser.add_argument('--learning_rate',
                        type=float, default=0.01,
                        help='Initial learning rate.')
    parser.add_argument('--num_epochs',
                        type=int, default=20,
                        help='Number of epochs to run trainer.')
    parser.add_argument('--batch_size',
                        type=int, default=10,
                        help='Batch size. Must divide evenly into the dataset sizes.')
    parser.add_argument('--log_dir',
                        type=str, default='logs',
                        help='Directory to put logging.')

    FLAGS, unparsed = parser.parse_known_args()

    print("LR: ", FLAGS.learning_rate)
    print("Batch size: ", FLAGS.batch_size)

    run_main(FLAGS)



LR:  0.01
Batch size:  10
Torch device selected:  cpu

=== Starting Training for Mode 1 ===

Epoch: 1 (Mode 1)
Train set: Average loss: 0.3807, Accuracy: 53521/60000 (89%)
Test set: Average loss: 0.3048, Accuracy: 9129/10000 (91%)

Epoch: 2 (Mode 1)
Train set: Average loss: 0.3095, Accuracy: 54754/60000 (91%)
Test set: Average loss: 0.3090, Accuracy: 9127/10000 (91%)

Epoch: 3 (Mode 1)
Train set: Average loss: 0.2981, Accuracy: 54933/60000 (92%)
Test set: Average loss: 0.2880, Accuracy: 9191/10000 (92%)

Epoch: 4 (Mode 1)
Train set: Average loss: 0.2906, Accuracy: 55062/60000 (92%)
Test set: Average loss: 0.2927, Accuracy: 9176/10000 (92%)

Epoch: 5 (Mode 1)
Train set: Average loss: 0.2859, Accuracy: 55202/60000 (92%)
Test set: Average loss: 0.2928, Accuracy: 9185/10000 (92%)

Epoch: 6 (Mode 1)
Train set: Average loss: 0.2834, Accuracy: 55151/60000 (92%)
Test set: Average loss: 0.2849, Accuracy: 9177/10000 (92%)

Epoch: 7 (Mode 1)
Train set: Average loss: 0.2808, Accuracy: 55220/60000 

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()

        # Shared convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)  # used only in model_3

        # Fully connected layers for each model
        self.fc1_model1 = nn.Linear(in_features=10 * 6 * 6, out_features=64)
        self.fc1_model2 = nn.Linear(in_features=40 * 6 * 6, out_features=64)
        self.fc1_model3 = nn.Linear(in_features=128 * 2 * 2, out_features=64)

        self.fc2 = nn.Linear(in_features=64, out_features=10)

        # Model 2 updated conv layers (bigger channels)
        self.conv1_model2 = nn.Conv2d(in_channels=3, out_channels=20, kernel_size=3)
        self.conv2_model2 = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3)

        # Model 3 updated conv layers (deeper)
        self.conv1_model3 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv2_model3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3_model3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)

        # Assign model mode
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        else:
            raise ValueError("Mode must be 1, 2, or 3.")

    def model_1(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2)
        X = torch.flatten(X, start_dim=1)
        X = F.relu(self.fc1_model1(X))
        X = self.fc2(X)
        return X

    def model_2(self, X):
        X = F.relu(self.conv1_model2(X))
        X = F.max_pool2d(X, 2)
        X = F.relu(self.conv2_model2(X))
        X = F.max_pool2d(X, 2)
        X = torch.flatten(X, start_dim=1)
        X = F.relu(self.fc1_model2(X))
        X = self.fc2(X)
        return X

    def model_3(self, X):
        X = F.relu(self.conv1_model3(X))
        X = F.max_pool2d(X, 2)
        X = F.relu(self.conv2_model3(X))
        X = F.max_pool2d(X, 2)
        X = F.relu(self.conv3_model3(X))
        X = F.max_pool2d(X, 2)
        X = torch.flatten(X, start_dim=1)
        X = F.relu(self.fc1_model3(X))
        X = self.fc2(X)
        return X


In [19]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
#from ConvNet import ConvNet
import argparse
import numpy as np

def train(model, device, train_loader, optimizer, criterion, epoch, batch_size):
    model.train()
    losses = []
    correct = 0
    for batch_idx, batch_sample in enumerate(train_loader):
        data, target = batch_sample
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    return train_loss, train_acc

def test(model, device, test_loader):
    model.eval()
    losses = []
    correct = 0
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.cross_entropy(output, target, reduction='mean')
            losses.append(loss.item())
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    return test_loss, accuracy

def run_main(FLAGS):
    # Check if cuda is available
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print("Torch device selected: ", device)

    # Create transformations for CIFAR-10
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load CIFAR-10 datasets
    dataset1 = datasets.CIFAR10('./data/', train=True, download=True, transform=transform)
    dataset2 = datasets.CIFAR10('./data/', train=False, transform=transform)
    train_loader = DataLoader(dataset1, batch_size=FLAGS.batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset2, batch_size=FLAGS.batch_size, shuffle=False, num_workers=4)

    # Dictionary to store results
    results = {}

    # Task 1: Test model_1 with different learning rates
    print("\n=== Task 1: Testing model_1 with different learning rates ===")
    for lr in [0.5, 0.1, 0.01]:
        print(f"\n--- Model 1, Learning Rate {lr} ---")
        model = ConvNet(mode=1).to(device)
        criterion = nn.CrossEntropyLoss(reduction='mean')
        optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=1e-7)
        best_accuracy = 0.0

        # Train for default 10 epochs
        for epoch in range(1, 11):  # Fixed 10 epochs
            print(f"\nEpoch: {epoch} (Model 1, LR={lr})")
            train_loss, train_accuracy = train(model, device, train_loader,
                                               optimizer, criterion, epoch, FLAGS.batch_size)
            test_loss, test_accuracy = test(model, device, test_loader)
            if test_accuracy > best_accuracy:
                best_accuracy = test_accuracy

        # Store result
        results[f"model_1_lr_{lr}"] = best_accuracy
        print(f"\nModel 1 (LR={lr}) - Best Test Accuracy: {best_accuracy:.2f}%")

    # Task 2: Test model_2 with different epochs
    print("\n=== Task 2: Testing model_2 with different epochs ===")
    for num_epochs in [10, 20, 30]:
        print(f"\n--- Model 2, Epochs {num_epochs} ---")
        model = ConvNet(mode=2).to(device)
        criterion = nn.CrossEntropyLoss(reduction='mean')
        optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-7)
        best_accuracy = 0.0

        # Train for specified number of epochs
        for epoch in range(1, num_epochs + 1):
            print(f"\nEpoch: {epoch} (Model 2, Epochs={num_epochs})")
            train_loss, train_accuracy = train(model, device, train_loader,
                                               optimizer, criterion, epoch, FLAGS.batch_size)
            test_loss, test_accuracy = test(model, device, test_loader)
            if test_accuracy > best_accuracy:
                best_accuracy = test_accuracy

        # Store result
        results[f"model_2_epochs_{num_epochs}"] = best_accuracy
        print(f"\nModel 2 (Epochs={num_epochs}) - Best Test Accuracy: {best_accuracy:.2f}%")

    # Print final results
    print("\n=== Final Results ===")
    print("Task 1 (Model 1, varying learning rates):")
    for lr in [0.5, 0.1, 0.01]:
        print(f"LR={lr}: Best Test Accuracy = {results[f'model_1_lr_{lr}']:.2f}%")
    print("\nTask 2 (Model 2, varying epochs):")
    for num_epochs in [10, 20, 30]:
        print(f"Epochs={num_epochs}: Best Test Accuracy = {results[f'model_2_epochs_{num_epochs}']:.2f}%")

if __name__ == '__main__':
    parser = argparse.ArgumentParser('CNN Exercise.')
    parser.add_argument('--batch_size',
                        type=int, default=100,
                        help='Batch size. Must divide evenly into the dataset sizes.')
    parser.add_argument('--log_dir',
                        type=str, default='logs',
                        help='Directory to put logging.')

    FLAGS, unparsed = parser.parse_known_args()

    print("Batch size: ", FLAGS.batch_size)

    run_main(FLAGS)

Batch size:  100
Torch device selected:  cpu

=== Task 1: Testing model_1 with different learning rates ===

--- Model 1, Learning Rate 0.5 ---

Epoch: 1 (Model 1, LR=0.5)
Train set: Average loss: 2.0239, Accuracy: 12956/50000 (26%)
Test set: Average loss: 1.7056, Accuracy: 3743/10000 (37%)

Epoch: 2 (Model 1, LR=0.5)
Train set: Average loss: 1.6730, Accuracy: 19715/50000 (39%)
Test set: Average loss: 1.5658, Accuracy: 4350/10000 (44%)

Epoch: 3 (Model 1, LR=0.5)
Train set: Average loss: 1.5516, Accuracy: 22354/50000 (45%)
Test set: Average loss: 1.5231, Accuracy: 4497/10000 (45%)

Epoch: 4 (Model 1, LR=0.5)
Train set: Average loss: 1.4841, Accuracy: 23489/50000 (47%)
Test set: Average loss: 1.7166, Accuracy: 4259/10000 (43%)

Epoch: 5 (Model 1, LR=0.5)
Train set: Average loss: 1.4120, Accuracy: 25068/50000 (50%)
Test set: Average loss: 1.3481, Accuracy: 5274/10000 (53%)

Epoch: 6 (Model 1, LR=0.5)
Train set: Average loss: 1.3442, Accuracy: 26288/50000 (53%)
Test set: Average loss: 1.4